In [1]:
1

1

calc total point 
from p4 j03 sumr tablation funcin usage iy

In [1]:
using JSON
using DataFrames
using DataFramesMeta
using Dates
using JLD2
using ShiftedArrays
using OrderedCollections
using CSV

In [2]:
# d_fs = "../dicts/ndb_master_creation-main/output_iy/"
# d_fs = "../../../storage/smr_iy_tabl_temp/"
d_fs = "../../../storage/smr_iy_tabl_fix/"
fs = readdir(d_fs)
fs

8-element Vector{String}:
 "smr_tbl_cancer_iy_dict_s_000_32_dpfix.jld2"
 "smr_tbl_cancer_iy_dict_s_001_32_dpfix.jld2"
 "smr_tbl_cancer_iy_dict_s_002_32_dpfix.jld2"
 "smr_tbl_cancer_iy_dict_s_003_32_dpfix.jld2"
 "smr_tbl_cancer_iy_dict_s_004_32_dpfix.jld2"
 "smr_tbl_cancer_iy_dict_s_005_32_dpfix.jld2"
 "smr_tbl_cancer_iy_dict_s_006_32_dpfix.jld2"
 "smr_tbl_cancer_iy_dict_s_007_32_dpfix.jld2"

In [4]:
f_iy = "../dicts/Jm01_dis_parse/dfmeta_iy_20231122_150053_929635.csv"
df_iy = @chain CSV.read(f_iy, header=1,types=Dict("c2"=>String), DataFrame) begin
    @transform(:yj2 = first.(:c31, 2))
    @subset(in.(:yj2, Ref(["42", "43"])))
    # @subset(in.(:c2, Ref(["662450004"])))
    # @subset(:c2 == "662450004")
end

const c2_limit = Set(df_iy[!, "c2"])
length(c2_limit)

1006

In [5]:
"620003762" in c2_limit

true

In [6]:

struct RangeDict
    d_1st
    d_dict
    d_meta
end


function get_rd(self::RangeDict, dt)
    # dt = Dates.Date(dt)
    res = self.d_1st
    for (k,v) in self.d_dict
        if dt < k
            break
        else
            res = copy(v)
        end
    end
    return res
end

f_rd = "../dicts/range_dict/range_dict_numkey.jld2"
rd_iy = load(f_rd)["data"]
get_rd(rd_iy["610406002"], 20140312)

Dict{String, Any} with 2 entries:
  "point" => 13.1
  "tag"   => "aes"

In [7]:
function gen_iy_dict()
    d_ym = Dict{Int32, Float32}()
    d_ymd = Dict{Int32, Float32}()
    for y in 2011:2022
        # println(string(y))
        for m in 1:12
            ym = string(y) * lpad(m, 2, "0")
            d_ym[parse(Int32, ym)] = 0.0
            # println(lpad(m, 2, "0"))
            for d in 1:31
                ymd = ym * lpad(d, 2, "0")
                d_ymd[parse(Int32, ymd)] = 0.0
                # println(lpad(d, 2, "0"))
            end
        end
    end

    f_iy = "../dicts/Jm01_dis_parse/dfmeta_iy_20231122_150053_929635.csv"
    df_iy = @chain CSV.read(f_iy, header=1,types=Dict("c2"=>String), DataFrame) begin
            end
    d_iy = Dict{String, Dict{Int32, Float32}}()
    for r in eachrow(df_iy)
        d_iy[r["c2"]] = copy(d_ymd)
    end
    
    
    println("base dicts generated")
    return  d_ym, d_ymd, d_iy
end

gen_iy_dict (generic function with 1 method)

In [8]:
d_ym, d_ymd, d_iy = gen_iy_dict()
print("")

base dicts generated


In [9]:
# d_iy

In [10]:
f_rdnk = "../dicts/range_dict/range_dict_numkey.jld2"
rd_iy = load(f_rdnk)["data"]

Dict{String, Any} with 31892 entries:
  "620703401" => RangeDict(Dict{String, Any}("point"=>5.6, "tag"=>"bdm"), Order…
  "612430006" => RangeDict(Dict{String, Any}("point"=>9.6, "tag"=>"bdm"), Order…
  "612590038" => RangeDict(Dict{String, Any}("point"=>8.4, "tag"=>"bdm"), Order…
  "610406252" => RangeDict(Dict{String, Any}("point"=>9.9, "tag"=>"bdm"), Order…
  "610443003" => RangeDict(Dict{String, Any}("point"=>51.9, "tag"=>"bdm"), Orde…
  "620006319" => RangeDict(Dict{String, Any}("point"=>181.0, "tag"=>"bdm"), Ord…
  "620005051" => RangeDict(Dict{String, Any}("point"=>5.7, "tag"=>"bdm"), Order…
  "620004178" => RangeDict(Dict{String, Any}("point"=>92.0, "tag"=>"bdm"), Orde…
  "622533201" => RangeDict(Dict{String, Any}("point"=>873.0, "tag"=>"bsl"), Ord…
  "610412050" => RangeDict(Dict{String, Any}("point"=>22.1, "tag"=>"bdm"), Orde…
  "610412077" => RangeDict(Dict{String, Any}("point"=>6.2, "tag"=>"bdm"), Order…
  "615100573" => RangeDict(Dict{String, Any}("point"=>1.75, "tag"=>"bdm

In [11]:

# get_rd(rd_iy["611240417"], 20140312)

In [12]:
# "610406002" 

In [11]:
get_rd(rd_iy["610406002"], 20140312)

Dict{String, Any} with 2 entries:
  "point" => 13.1
  "tag"   => "aes"

In [12]:
# tmp_ = 0
d_use_whole_dose = Dict{String, Dict{String, Dict{Int32, Float32}}}()

for f in fs
    a = now()
    f2= d_fs*f
    println(f)
    d_ = load(f2)["data"]
    # tmp_ = d_
    for (cancer, used_drugs) in d_
        if !haskey(d_use_whole_dose, cancer)
            # d_use_whole_dose[cancer] = Dict{String, Dict{Int32, Float32}}
            d_use_whole_dose[cancer] = copy(d_iy)
        end
        for (drug_code, ymd_usage) in used_drugs
            if drug_code in c2_limit
                for (ymd, usage) in ymd_usage
                    d_use_whole_dose[cancer][drug_code][ymd] += usage
                end
            end
        end

    end
    # break
    b = now()
    print(" time:")
    println(canonicalize(b -a))    
end

smr_tbl_cancer_iy_dict_s_000_32_dpfix.jld2
 time:5 minutes, 1 second, 841 milliseconds
smr_tbl_cancer_iy_dict_s_001_32_dpfix.jld2
 time:4 minutes, 36 seconds, 208 milliseconds
smr_tbl_cancer_iy_dict_s_002_32_dpfix.jld2
 time:4 minutes, 44 seconds, 939 milliseconds
smr_tbl_cancer_iy_dict_s_003_32_dpfix.jld2
 time:4 minutes, 36 seconds, 150 milliseconds
smr_tbl_cancer_iy_dict_s_004_32_dpfix.jld2
 time:4 minutes, 42 seconds, 936 milliseconds
smr_tbl_cancer_iy_dict_s_005_32_dpfix.jld2
 time:4 minutes, 35 seconds, 464 milliseconds
smr_tbl_cancer_iy_dict_s_006_32_dpfix.jld2
 time:4 minutes, 43 seconds, 26 milliseconds
smr_tbl_cancer_iy_dict_s_007_32_dpfix.jld2
 time:4 minutes, 46 seconds, 112 milliseconds


In [13]:
d_use_whole_dose

Dict{String, Dict{String, Dict{Int32, Float32}}} with 63 entries:
  "Bone"                        => Dict("620703401"=>Dict(20110131=>0.0, 201202…
  "Tongue"                      => Dict("620703401"=>Dict(20110131=>0.0, 201202…
  "Connective and soft tissue"  => Dict("620703401"=>Dict(20110131=>0.0, 201202…
  "Stomach"                     => Dict("620703401"=>Dict(20110131=>0.0, 201202…
  "Other endocrine"             => Dict("620703401"=>Dict(20110131=>0.0, 201202…
  "Liver"                       => Dict("620703401"=>Dict(20110131=>0.0, 201202…
  "Other urinary organs"        => Dict("620703401"=>Dict(20110131=>0.0, 201202…
  "Prostate"                    => Dict("620703401"=>Dict(20110131=>0.0, 201202…
  "Other female genital organs" => Dict("620703401"=>Dict(20110131=>0.0, 201202…
  "Hypopharynx"                 => Dict("620703401"=>Dict(20110131=>0.0, 201202…
  "Salivary glands"             => Dict("620703401"=>Dict(20110131=>0.0, 201202…
  "Leukaemia unspecified"       => Dict("62

In [14]:
f_out ="../../../storage/smr_iy_res/iy_cost_dict_whole_fix.jld2"
save(f_out,"data", d_use_whole_dose)

In [ ]:
kokomade